In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import numpy as np
from pprint import pformat
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

## Strategy

In [ ]:
# Create a Stratey
class OptStrategy(bt.Strategy):
    params = (
        ('emaSperiod', 7),
        ('emaLperiod', 15),
        ('cross_fac', 1.0),
        ('printlog', False),
        ('printstop', False)
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        self.ema_short = bt.indicators.ExponentialMovingAverage(
                            self.datas[0], period=self.params.emaSperiod)
        self.ema_long = bt.indicators.ExponentialMovingAverage(
                            self.datas[0], period=self.params.emaLperiod)     
        self.xf = self.params.cross_fac

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.ema_short[0]*self.xf < self.ema_long[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.ema_short[0]*self.xf >= self.ema_long[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('ema params:(%2d, %2d, %.2d), %.2f' %
                 (self.params.emaSperiod, self.params.emaLperiod, self.params.cross_fac, self.broker.getvalue()), doprint=self.params.printstop)

## Analyzers

In [ ]:
class DecisiveAnalyzer(bt.Analyzer):
    
    def __init__(self):
        self.emaSperiod = self.strategy.params.emaSperiod
        self.emaLperiod = self.strategy.params.emaLperiod
        self.xf = self.strategy.params.cross_fac
        self.equity = []
        
    def start(self):
        # Not needed ... but could be used
        self.init_cash = self.strategy.broker.cash 
        self.ntrade = 0

    def next(self):
        # Not needed ... but could be used
        pass

    def notify_trade(self, trade):

        if trade.isclosed:
            self.equity.append(self.strategy.broker.getvalue())
            self.ntrade += 1
            
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        equity = np.asarray([self.init_cash,] + self.equity)

        outp = {
            'params': (self.emaSperiod, self.emaLperiod, self.xf),
            'profit': self.final_val - self.init_cash,
            'ntrade': self.ntrade,
            'trades': np.diff(equity).tolist(),
        }
        
        return outp
    

In [ ]:
def best_result_from_cerebro_opti_run(result):
    params  = []
    n_trades = []
    profit   = []
    trades   = []
    for res in result:
        r = res[0].analyzers.decisive.get_analysis()
        params.append(r['params'])
        n_trades.append(r['ntrade'])
        profit.append(r['profit'])
        trades.append(r['trades'])

    prof_ind = np.argmax(profit) 
    best_params = params[prof_ind]
    best_profit = profit[prof_ind]
    best_ntrades = n_trades[prof_ind]
    best_trades = trades[prof_ind]
    
    #print('best:{} profit:{} trades:{}'.format(best_params, best_profit, best_ntrades))
    return (best_params, best_profit, best_ntrades, best_trades)

In [ ]:
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()
df['2014-01-01':'2019-06-01']['close'].plot()

In [ ]:
df.index[0] + datetime.timedelta(days=3)

## In-sample Parameters

In [ ]:
def get_wf_dates(data, nday_train, nday_test):
    periods = []
    start = data.index[0]
    end = data.index[-1]

    n = 0
    is_start  = start
    while (end - is_start).days >= (nday_train + nday_test):

        is_end = is_start + dt.timedelta(days=nday_train)
        oos_start = is_end
        oos_end = oos_start + dt.timedelta(days=nday_test)

        periods.append({
                        'run': n, 
                        'oos': (oos_start, oos_end), 
                        'is': (is_start, is_end)
                        })
        n += 1
        is_start += dt.timedelta(days=nday_test)
    
    return periods

In [ ]:
nday_train = 365
nday_test  = 90
periods = get_wf_dates(df, nday_train, nday_test)
    
best_oos_params = []
for oos_is in periods:
    start_date, end_date = oos_is['is']
    run_num = oos_is['run']
    #print('RUN {} \t START/END: {}/{}'.format(run_num, start_date, end_date))
    cerebro = bt.Cerebro()

    strats = cerebro.optstrategy(
                OptStrategy,
                emaSperiod=range(3, 6, 1),
                emaLperiod=range(10, 14, 1),
                cross_fac=1.0)
    
    cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

    cerebro.optreturn = False
    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)
    cerebro.broker.set_slippage_fixed(.00, 
                                  slip_open=True, 
                                  slip_limit=True, 
                                  slip_match=True, 
                                  slip_out=True)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    best_param, best_profit, best_trades, best_equity = best_result_from_cerebro_opti_run(result)

    best_oos_params.append({'train_param': best_param, 
                                      'train_profit': best_profit, 
                                      'train_numtrades': best_trades, 
                                      'train_tradeslist': best_equity, 
                                      'train_period': oos_is['is'],
                                      'test_period': oos_is['oos'],
                                     })
    
    #print('')

print('The best OOS parameters are:')
for sim in best_oos_params:
    print(pformat(sim['test_period']), pformat(sim['train_param']), pformat(sim['train_numtrades']), pformat(sim['train_profit']))

In [ ]:
for sim in best_oos_params:
    trades_list = np.asarray(sim['train_tradeslist'])
    cumsum = trades_list.cumsum()
    plt.plot(cumsum)

## Out-of-sample Parameters

In [ ]:
run_num = 0
for best in best_oos_params:
    start_date, end_date = best['test_period']
    print('RUN {} \t START/END: {}/{}'.format(run_num, start_date, end_date))
    cerebro = bt.Cerebro()

    cerebro.addstrategy(OptStrategy, 
                        emaSperiod=best['train_param'][0],
                        emaLperiod=best['train_param'][1],
                        cross_fac=best['train_param'][2])

    
    cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    r = result[0].analyzers.decisive.get_analysis()
    best_oos_params[run_num]['test_numtrades'] = r['ntrade']
    best_oos_params[run_num]['test_tradeslist'] = r['trades']
    best_oos_params[run_num]['test_profit'] = r['profit']
    run_num += 1

    

In [ ]:
print('The best OOS parameters are:')
for sim in best_oos_params:
    print((sim['test_period']), (sim['train_param']), (sim['train_numtrades']), (sim['train_profit']), (sim['test_numtrades']), (sim['test_profit']))
    

In [ ]:
def days_from_date_tuple(mytuple):
    insample_days_start = mytuple[0]
    insample_days_end = mytuple[1]
    insample_days = (insample_days_end - insample_days_start).days
    return insample_days

oos_trades = []
wfe_list = []
run_num=0
for values in best_oos_params:
    oos_trades_list = values['test_tradeslist']
    oos_trades.extend(oos_trades_list)
    
    insample_days = days_from_date_tuple(values['train_period'])
    oos_days = days_from_date_tuple(values['test_period'])

    insample_annual_profit = 365/insample_days * values['train_profit']
    oos_annual_profit = 365/oos_days * values['test_profit']

    walkforward_efficiency_pct = round(oos_annual_profit * 100 / insample_annual_profit, 1)
    
    wfe_list.append(walkforward_efficiency_pct)

    print('Run {} WFE:{}'.format(run_num, walkforward_efficiency_pct))
    print('\t IS_DAYS: {} OOS_DAYS: {} IS_PROFIT: {:0.2f} OOS_PROFIT: {:0.2f}'.format(insample_days, oos_days,
                                                                            insample_annual_profit, oos_annual_profit))
    run_num += 1
    
avg_wfe = np.round(np.asarray(wfe_list).mean(),1)
print('Average WFE: {}%'.format(avg_wfe))


## Walkforward

In [ ]:
# Create a Stratey
class WalkforwardStrategy(OptStrategy):
    params = (
        ('emaSperiod', 15),
        ('emaLperiod', 30),
        ('cross_fac', 1.0),
        ('printlog', False),
        ('printstop', False),
        ('live', False),
        ('walkforward', None),
    )


    def __init__(self):
        super().__init__()
        
        # Trim the indicators if we are running live, just need the current one
        if not self.params.live:
            pass
        
        self.current_row = None
        self.wfema = []
        
        if self.params.walkforward:
            for run in self.params.walkforward:
                self.wfema.append({
                    'emaS': bt.indicators.ExponentialMovingAverage(self.datas[0], period=run['train_param'][0]),
                    'emaL': bt.indicators.ExponentialMovingAverage(self.datas[0], period=run['train_param'][1]),
                    'xf': run['train_param'][2],
                    'test_period': run['test_period'],
                    'train_param': run['train_param'],
                }) 

        print(pformat(self.wfema))

    def next(self):

        # Walk-forward logic
        for row in self.wfema:
            start, end = row['test_period']
            period_start = start
            period_end = end
            if self.datetime.datetime() >= period_start and self.datetime.datetime() < period_end:
                self.current_row = row
        
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.current_row['emaS'][0]*self.current_row['xf'] \
                                        < self.current_row['emaL'][0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.current_row['emaS'][0]*self.current_row['xf'] \
                                        >= self.current_row['emaL'][0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()


In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(WalkforwardStrategy, 
                    emaSperiod=7,
                    emaLperiod=30,
                    cross_fac=1.0,
                    walkforward=best_oos_params,
                    live=False)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# periods
start_date = best_oos_params[0]['test_period'][0]
end_date = best_oos_params[-1]['test_period'][0]

print('Start: {} End: {}'.format(start_date, end_date))
data = bt.feeds.PandasData(dataname = df[start_date:end_date])


# Add the Data Feed to Cerebro
cerebro.adddata(data)
cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
r = results[0].analyzers.decisive.get_analysis()
cumsum = np.asarray(r['trades']).cumsum()

plt.figure(figsize=(10,5))
plt.plot(cumsum)
plt.title('Walkforward Equity Curve')
plt.xlabel('Trades')
plt.ylabel('Equity')

print('Average WFE: {}%'.format(avg_wfe))

# cerebro.plot(volume=False, iplot=False)


In [ ]:
cerebro.plot()